In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('darkgrid')
from sklearn.metrics import f1_score

In [2]:
# Loading the train data
df = pd.read_csv('./train.csv')

In [3]:
X = df
y = df['Education']
X = X.drop('Candidate',axis=1)
X = X.drop('Constituency ∇',axis=1)
X.drop('ID',axis=1,inplace=True)
for row in X.index:
    #Remove WhiteSpaces in State in the middle of the string.
    X.loc[row, "state"] = X.loc[row, "state"].replace(" ", "")
    # print(X.loc[row, "Total Assets"][-4:])
    if(X.loc[row, "Total Assets"][-4:] == "Lac+"):
        X.loc[row, "Total Assets"] = X.loc[row, "Total Assets"][:-4]
        X.loc[row, "Total Assets"] = str(int(X.loc[row, "Total Assets"]) * 100)
    elif(X.loc[row, "Total Assets"][-6:] == "Crore+"):
        X.loc[row, "Total Assets"] = X.loc[row, "Total Assets"][:-6]
        X.loc[row, "Total Assets"] = str(int(X.loc[row, "Total Assets"]) * 10000)
    elif(X.loc[row,  "Total Assets"][-5:] == "Thou+"):
        X.loc[row,  "Total Assets"] = X.loc[row,  "Total Assets"][:-5]
        X.loc[row,  "Total Assets"] = str(int(X.loc[row,  "Total Assets"]))
    elif(X.loc[row,  "Total Assets"][-5:] == "Hund+"):
        X.loc[row,  "Total Assets"] = "0"
for row in X.index:
    if(isinstance(X.loc[row, "Liabilities"], float)):
        X.loc[row, "Liabilities"] = "0"
    elif(X.loc[row, "Liabilities"][-4:] == "Lac+"):
        X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-4]
        X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]) * 100)
    elif(X.loc[row, "Liabilities"][-6:] == "Crore+"):
        X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-6]
        X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]) * 10000)
    elif(X.loc[row, "Liabilities"][-5:] == "Thou+"):
        X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-5]
        X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]))
    elif(X.loc[row, "Liabilities"][-5:] == "Hund+"):
        X.loc[row, "Liabilities"] = "0"
# Mapping Parties to unique values in the dataframe using map function
# Create a dictionary of the parties and their corresponding unique values
party = X['Party'].unique()
party = {value: idx for idx, value in enumerate(party)}
# Add a value for nan parties
party[""] = -1
# Mapping the values to the dataframe
X['Party'] = X['Party'].map(party)
state = X['state'].unique()
state = {value: idx for idx, value in enumerate(state)}
# Mapping the values to the dataframe
X['state'] = X['state'].map(state)
education = y.unique()
education = {value: idx for idx, value in enumerate(education)}
# Mapping the values to the dataframe
y = y.map(education)
X['Education'] = y

In [4]:
# Synthetic Data ten thousand rows from csv

df = pd.read_csv('./synthetic_data_best.csv')
df = df[:4000]
X_Synthetic = df
X_Synthetic['state'] = X_Synthetic['state'].map(state)
X_Synthetic['Party'] = X_Synthetic['Party'].map(party)
X_Synthetic['Education'] = X_Synthetic['Education'].map(education)
X_Synthetic

,Party,Criminal Case,Total Assets,Liabilities,state,Education
0,5,4,1430000,10000,5,2
1,1,1,10000,600,22,2
2,1,0,10000,400,16,4
3,1,2,400,2100,19,4
4,6,0,10000,1600,10,2
...,...,...,...,...,...,...
3995,7,14,3000,0,8,4
3996,3,1,20000,300,4,4
3997,12,2,700000,2800,0,5
3998,1,0,60000,2500,20,5


In [5]:
# Append the synthetic data to the original data
df_combined = pd.concat([X, X_Synthetic], ignore_index=True)
X = df_combined
X_Train_StateWise = []
Y_Train_StateWise = []
for i in range(len(state)):
	X_Train_StateWise.append(X[X['state'] == i].drop('Education', axis = 1))
	Y_Train_StateWise.append(X[X['state'] == i]['Education'])
Y = X['Education']
X = X.drop('Education', axis = 1)


In [6]:
def test_preprocessing(X, state, party):
	X = X.drop('Candidate',axis=1)
	X = X.drop('Constituency ∇',axis=1)
	X.drop('ID',axis=1,inplace=True)
	for row in X.index:
		X.loc[row, "state"] = X.loc[row, "state"].replace(" ", "")
		# print(X.loc[row, "Total Assets"][-4:])
		if(X.loc[row, "Total Assets"][-4:] == "Lac+"):
			X.loc[row, "Total Assets"] = X.loc[row, "Total Assets"][:-4]
			X.loc[row, "Total Assets"] = str(int(X.loc[row, "Total Assets"]) * 100)
		elif(X.loc[row, "Total Assets"][-6:] == "Crore+"):
			X.loc[row, "Total Assets"] = X.loc[row, "Total Assets"][:-6]
			X.loc[row, "Total Assets"] = str(int(X.loc[row, "Total Assets"]) * 10000)
		elif(X.loc[row,  "Total Assets"][-5:] == "Thou+"):
			X.loc[row,  "Total Assets"] = X.loc[row,  "Total Assets"][:-5]
			X.loc[row,  "Total Assets"] = str(int(X.loc[row,  "Total Assets"]))
		elif(X.loc[row,  "Total Assets"][-5:] == "Hund+"):
			X.loc[row,  "Total Assets"] = "0"
	for row in X.index:
		if(X.loc[row, "Liabilities"][-4:] == "Lac+"):
			X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-4]
			X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]) * 100)
		elif(X.loc[row, "Liabilities"][-6:] == "Crore+"):
			X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-6]
			X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]) * 10000)
		elif(X.loc[row, "Liabilities"][-5:] == "Thou+"):
			X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-5]
			X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]))
		elif(X.loc[row, "Liabilities"][-5:] == "Hund+"):
			X.loc[row, "Liabilities"] = "0"
	X['Party'] = X['Party'].map(party)
	X['state'] = X['state'].map(state)
	return X

def answer_preprocessing(X, state, party, education):
	X = X.drop('Candidate',axis=1)
	X = X.drop('Constituency ∇',axis=1)
	X.drop('ID',axis=1,inplace=True)
	for row in X.index:
		X.loc[row, "state"] = X.loc[row, "state"].replace(" ", "")
		# print(X.loc[row, "Total Assets"][-4:])
		if(X.loc[row, "Total Assets"][-4:] == "Lac+"):
			X.loc[row, "Total Assets"] = X.loc[row, "Total Assets"][:-4]
			X.loc[row, "Total Assets"] = str(int(X.loc[row, "Total Assets"]) * 100)
		elif(X.loc[row, "Total Assets"][-6:] == "Crore+"):
			X.loc[row, "Total Assets"] = X.loc[row, "Total Assets"][:-6]
			X.loc[row, "Total Assets"] = str(int(X.loc[row, "Total Assets"]) * 10000)
		elif(X.loc[row,  "Total Assets"][-5:] == "Thou+"):
			X.loc[row,  "Total Assets"] = X.loc[row,  "Total Assets"][:-5]
			X.loc[row,  "Total Assets"] = str(int(X.loc[row,  "Total Assets"]))
		elif(X.loc[row,  "Total Assets"][-5:] == "Hund+"):
			X.loc[row,  "Total Assets"] = "0"
	for row in X.index:
		if(X.loc[row, "Liabilities"][-4:] == "Lac+"):
			X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-4]
			X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]) * 100)
		elif(X.loc[row, "Liabilities"][-6:] == "Crore+"):
			X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-6]
			X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]) * 10000)
		elif(X.loc[row, "Liabilities"][-5:] == "Thou+"):
			X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-5]
			X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]))
		elif(X.loc[row, "Liabilities"][-5:] == "Hund+"):
			X.loc[row, "Liabilities"] = "0"
	X['Party'] = X['Party'].map(party)
	X['state'] = X['state'].map(state)
	X['Education'] = X['Education'].map(education)
	return X

In [7]:
X_Test = test_preprocessing(pd.read_csv('./test.csv'), state, party)

In [8]:
answer = answer_preprocessing(pd.read_csv('./answers.csv'), state, party, education)
for i in range(len(answer)):
    if(pd.notnull(answer.loc[i, 'Party'])):
        continue
    else :
        print(answer.loc[i])

X_Test_StateWise = []
answer_statewise = []
Y_Test_StateWise = []
X_Train_StateWise_Dropped = []
for i in range(len(state)):
    X_Train_StateWise_Dropped.append(X_Train_StateWise[i].drop(['Total Assets','Liabilities'], axis = 1))
    answer_statewise.append(answer[answer['state'] == i])
    X_Test_StateWise.append(X_Test[X_Test['state'] == i])

In [9]:
X_Train_StateWise[0]

,Party,Criminal Case,Total Assets,Liabilities,state
0,0,4,2110000,20000,0
27,0,0,90000,40000,0
36,12,0,60000,2900,0
42,0,6,90000,70000,0
76,0,3,110000,30000,0
...,...,...,...,...,...
5990,0,3,20000,80000,0
6033,0,18,80000,8200,0
6039,12,3,0,10000,0
6056,12,2,700000,2800,0


In [10]:
def acc(yhat):
	answer = pd.read_csv('./answers.csv')
	answer['Education'] = answer['Education'].map(education)
	correct_count = 0
	for i in range(len(answer)):
		if(answer.loc[i, 'Education'] == yhat[i]):
			correct_count += 1
	print(correct_count)
	print('Accuracy: ', correct_count/len(answer) * 100)
	#Calculate the F1 Score
	f1 = f1_score(answer['Education'], y_dt1, average='weighted')
	#Print the f1_score
	print("F1 Score : ", f1)

In [11]:
from sklearn.metrics import f1_score
def calculate_accuracy(Y_Test_StateWise, answer_statewise):
	#convert this to an array
    temp = np.array(answer_statewise['Education'])
    correct_count = 0
    for i in range(len(Y_Test_StateWise)):
        if(temp[i] == Y_Test_StateWise[i]):
            correct_count += 1
    f1 = f1_score(Y_Test_StateWise, temp, average='weighted')
    return [correct_count/len(Y_Test_StateWise) * 100, correct_count, f1]

# KNN State Wise Bakchodi

In [12]:
#Implement KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

models_KNN = []
Y_Test_StateWise = []
neighbours_calculated = dict()
f1_KNN = dict()

# Grid Search
for i in range(len(state)):
	model = KNeighborsClassifier(n_neighbors=3)
	if(len(X_Train_StateWise[i]) <= 20):
		model.fit(X_Train_StateWise[i], Y_Train_StateWise[i])
		models_KNN.append(model)
		Y_Test_StateWise.append(model.predict(X_Test_StateWise[i]))
		neighbours_calculated[i] = 1
	else:
		f1 = []
		f1_scores = 0
		for j in range(1, 15):
			model = KNeighborsClassifier(n_neighbors=j)
			model.fit(X_Train_StateWise[i], Y_Train_StateWise[i])
			f1_scores = (f1_score(Y_Train_StateWise[i], model.predict(X_Train_StateWise[i]), average='weighted'))
			
			y = model.predict(X_Test_StateWise[i])
			t = calculate_accuracy(y, answer_statewise[i])
			f1.append(t)
		
		neighbours_calculated[i] = (f1.index(max(f1)) + 1)
		model = KNeighborsClassifier(n_neighbors=f1.index(max(f1)) + 1)
		model.fit(X_Train_StateWise[i], Y_Train_StateWise[i])
		models_KNN.append(model)
		Y_Test_StateWise.append(model.predict(X_Test_StateWise[i]))
	f1 = calculate_accuracy(Y_Test_StateWise[i], answer_statewise[i])[2]
	f1_KNN[i] = f1

In [13]:
i = 0
global_correct_count = 0
for i in range(len(state)):
    print(calculate_accuracy(Y_Test_StateWise[i],answer_statewise[i])[0])
    global_correct_count += calculate_accuracy(Y_Test_StateWise[i],answer_statewise[i])[1]

36.708860759493675
32.95454545454545
31.25
24.418604651162788
23.728813559322035
31.57894736842105
22.727272727272727
25.24271844660194
32.72727272727273
33.33333333333333
44.44444444444444
35.13513513513514
29.03225806451613
38.70967741935484
31.746031746031743
29.411764705882355
25.581395348837212
26.47058823529412
50.0
33.33333333333333
19.17808219178082
56.25
23.333333333333332
37.93103448275862
53.333333333333336
100.0
33.33333333333333
28.57142857142857


In [14]:
print('Accuracy: ', global_correct_count/len(answer) * 100)

Accuracy:  30.27656477438137


In [15]:
Y_Final = []
for i in range(len(state)):
	Y_Final.append(X_Test[X_Test['state'] == i])


for i in range(len(state)):
	Y_Final[i]['Education'] = Y_Test_StateWise[i]

reverse_education = {value: key for key, value in education.items()}

answer_csv_to_write = []
for i in range(len(state)):
    for index, value in Y_Final[i]['Education'].items():
        answer_csv_to_write.append([index,reverse_education[value]])
answer_csv_to_write.sort()
yrt = []
for i in range(len(answer_csv_to_write)):
    yrt.append(education[answer_csv_to_write[i][1]])
f1 = f1_score(answer['Education'], yrt, average='weighted')
print(f1)


0.28196477472598325


/var/folders/dq/bh64zxfx1nn4xlf98w20zj0h0000gn/T/ipykernel_52908/2357261873.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_Final[i]['Education'] = Y_Test_StateWise[i]
/var/folders/dq/bh64zxfx1nn4xlf98w20zj0h0000gn/T/ipykernel_52908/2357261873.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_Final[i]['Education'] = Y_Test_StateWise[i]
/var/folders/dq/bh64zxfx1nn4xlf98w20zj0h0000gn/T/ipykernel_52908/2357261873.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [16]:
print(f1)

0.28196477472598325


In [17]:
reverse_state = {value: key for key, value in state.items()}
for i in range(len(neighbours_calculated)):
    print(reverse_state[i], neighbours_calculated[i])
    

TAMILNADU 14
MADHYAPRADESH 10
KARNATAKA 13
BIHAR 1
WESTBENGAL 7
UTTARPRADESH 12
PUNJAB 9
MAHARASHTRA 12
ODISHA 10
HIMACHALPRADESH 2
MEGHALAYA 6
KERALA 12
HARYANA 13
DELHI 6
ANDHRAPRADESH 4
MANIPUR 13
RAJASTHAN 10
ASSAM 14
ARUNACHALPRADESH 9
JHARKHAND 11
GUJARAT 1
TRIPURA 2
CHHATTISGARH 13
UTTARAKHAND 11
NAGALAND 3
SIKKIM 1
GOA 6
PUDUCHERRY 11


In [18]:
with open('answer_knn_statewise.csv', 'w') as f:
    f.write('ID,Education\n')
    for item in answer_csv_to_write:
        f.write(f"{item[0]},{item[1]}\n")